# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')

import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import multioutput
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
nltk.download('stopwords')
#from custom_transformer import StartingVerbExtractor
from sklearn.metrics import fbeta_score, make_scorer
import pickle
from sklearn.multioutput import MultiOutputClassifier
import datetime

import matplotlib.pyplot as plt
%matplotlib inline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql ('SELECT * FROM MessagesCategories', engine)
X = df ['message']
Y = df.iloc[:,4:]
display (Y.head (n=3))

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [22]:
def tokenize(text):
    # Convert to lowercase
    text = text.lower() 
     # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    stop_words = stopwords.words("english")
    
    #tokenize
    words = word_tokenize (text)
    
    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    #lemmatizing
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed if w not in stop_words]
   
    return lemmed
    

In [23]:
print("text:",X[1],"\ntokens:", tokenize(X[1]))

text: Is the Hurricane over or is it not over 
tokens: ['hurrican']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', multioutput.MultiOutputClassifier (RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
start_time  = datetime.datetime.now()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 35)

In [32]:
#Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [33]:
end_time = datetime.datetime.now()
print ('Training of Pipeline Done..., Time: %d' % ((end_time - start_time).seconds/60),'minutes')

Training of Pipeline Done..., Time: 2 minutes


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [37]:
y_pred_train=pipeline.predict(X_train)

ValueError: Can only compare identically-labeled DataFrame objects

In [58]:
print(classification_report(y_train,y_pred_train, target_names= Y.columns))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     14857
               request       0.99      0.87      0.93      3348
                 offer       1.00      0.68      0.81        87
           aid_related       0.99      0.94      0.97      8155
          medical_help       1.00      0.76      0.86      1573
      medical_products       0.99      0.71      0.83       965
     search_and_rescue       1.00      0.73      0.84       533
              security       1.00      0.72      0.84       351
              military       1.00      0.73      0.85       644
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.76      0.86      1271
                  food       1.00      0.79      0.88      2232
               shelter       1.00      0.77      0.87      1762
              clothing       1.00      0.68      0.81       307
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [59]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa7f246f950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [60]:
parameters = {   
                  'tfidf__use_idf': [True]                 
                 
              }

In [61]:
cv = GridSearchCV(pipeline,param_grid=parameters, verbose = 10)

In [62]:
model=cv.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] tfidf__use_idf=True .............................................
[CV] .... tfidf__use_idf=True, score=0.1434715821812596, total= 1.3min
[CV] tfidf__use_idf=True .............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV] ... tfidf__use_idf=True, score=0.14055299539170507, total= 1.3min
[CV] tfidf__use_idf=True .............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.5min remaining:    0.0s


[CV] .... tfidf__use_idf=True, score=0.1468735596865878, total= 1.3min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.3min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [63]:
y_pred_train = cv.predict(X_train)
print(classification_report(y_train.values, y_pred_train, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.98      1.00      0.99     14857
               request       0.99      0.88      0.93      3348
                 offer       1.00      0.71      0.83        87
           aid_related       1.00      0.93      0.96      8155
          medical_help       1.00      0.75      0.86      1573
      medical_products       0.99      0.73      0.84       965
     search_and_rescue       1.00      0.75      0.85       533
              security       0.99      0.73      0.84       351
              military       1.00      0.74      0.85       644
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.74      0.85      1271
                  food       0.99      0.78      0.88      2232
               shelter       1.00      0.76      0.86      1762
              clothing       1.00      0.73      0.85       307
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [57]:
cv.best_params_

{'tfidf__use_idf': True}

In [66]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', multioutput.MultiOutputClassifier (RandomForestClassifier()))
        ])

parameters2 = {'vect__min_df': [5],
               'vect__max_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__bootstrap':[True]
              }

cv2 = GridSearchCV(pipeline2, param_grid = parameters2,  verbose = 10)

In [67]:
np.random.seed(35)
model2 = cv2.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__bootstrap=True, tfidf__use_idf=True, vect__max_df=5, vect__min_df=5 
[CV]  clf__estimator__bootstrap=True, tfidf__use_idf=True, vect__max_df=5, vect__min_df=5, score=0.17772657450076804, total=  50.8s
[CV] clf__estimator__bootstrap=True, tfidf__use_idf=True, vect__max_df=5, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__bootstrap=True, tfidf__use_idf=True, vect__max_df=5, vect__min_df=5, score=0.17480798771121353, total=  53.0s
[CV] clf__estimator__bootstrap=True, tfidf__use_idf=True, vect__max_df=5, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.6min remaining:    0.0s


[CV]  clf__estimator__bootstrap=True, tfidf__use_idf=True, vect__max_df=5, vect__min_df=5, score=0.17821477953602705, total=  52.3s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.0min finished


### 9. Export your model as a pickle file

In [68]:
pickle.dump(model, open("disaster_Response_model.pkl",'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.